# ALM 1 Ingest

The present tutorial uses the ALM-1 data set to demonstrate how to build and use a DataJoint pipeline.

The dataset is described here: https://crcns.org/data-sets/motor-cortex/alm-1/about-alm-1

The data structure is described here: https://crcns.org/files/data/alm-1/crcns_alm-1_data_description.pdf

## Install datajoint
First, make sure that datajoint is installed and that you can import it.  The installation instructions are found at http://docs.datajoint.io/setup/Install-and-connect.html

Then, import:

In [1]:
import datajoint as dj
import scio
import 

## Connect to your database server

The first thing you will need to do is to obtain your database credentials.  Contact your database administrator for this information.  Set the credentials using the `dj.config` dictionary:

In [2]:
dj.config['database.host'] = 'mesoscale-activity.datajoint.io'
dj.config['database.user'] = 'dimitri'       # substitute your username

## Create or connect to an existing database
Next specify which `database` (aka _schema_) you will be working with.  Each data pipeline may comprise multiple schemas for different portions of the pipeline.  

In in this tutorial we will work with only one schema called `tutorial_alm1`.  The following command will create the database if it does not already exist.  It also returns the object (called `schema`) that will be used to associate Python objects with tables in this database.  

In [3]:
schema = dj.schema('tutorial_alm1', locals())

Please enter DataJoint password: ········
Connecting dimitri@mesoscale-activity.datajoint.io:3306


# Import Animal and Session data

In [27]:
import os
import scipy.io as scio

meta_data_path = 'data/ALM-1/meta_data'

@schema
class SessionDirectory(dj.Lookup):

    definition = """
    session_file : varchar(255)    
    """
    
    contents = [[os.path.join(meta_data_path, f)]
                for f in os.listdir(meta_data_path) if f.endswith('.mat')]


In [28]:
@schema
class Animal(dj.Imported):
    definition = """
    animal :  int  # animal id 
    ---
    species  : varchar(255)
    date_of_birth : date
    """
    
    class GeneModification(dj.Part):
        definition = """
        -> Animal
        gene_modification : varchar(30)
        """
        
    class Strain(dj.Part):
        definition = """
        -> Animal
        strain  : varchar(30)
        """


@schema
class Session(dj.Imported):
    definition = """
    -> Animal
    session : tinyint 
    ---
    session_date   :  date
    session_suffix :  char(1) 
    -> SessionDirectory
    UNIQUE INDEX(animal, session_date, session_suffix)
    """

    @property
    def key_source(self):
        return SessionDirectory()
    
    def _make_tuples(self, key):        
        r = scio.loadmat(key['session_file'], 
                         struct_as_record=False, squeeze_me=True)['meta_data']
        # extract animalID
        if not isinstance(r.animalID, str):
            r.animalID = r.animalID[0]   # handles errors in data
        animal = int(r.animalID[3:])  
        key['animal'] = animal
        
        # insert animal if first time
        if not (Animal() & key):
            tup = dict(
                animal=animal,
                species=r.species, 
                date_of_birth = "{year}-{month}-{day}".format(
                    year=r.dateOfBirth[0:4], 
                    month=r.dateOfBirth[4:6], 
                    day=r.dateOfBirth[6:8]))
            Animal().insert1(tup)
            if isinstance(r.animalGeneModification, str):
                r.animalGeneModification = [r.animalGeneModification]
            Animal.GeneModification().insert(
                dict(animal=animal, gene_modification=m) for m in r.animalGeneModification)
            if isinstance(r.animalStrain, str):
                r.animalStrain = [r.animalStrain]
            Animal.Strain().insert(
                dict(animal=animal, strain=m) for m in r.animalStrain)

        tup = key
        tup['session'] = len(Session() & dict(animal=animal))+1  
        tup['session_date'] = "{year}-{month}-{day}".format(
                year=r.dateOfExperiment[0:4], 
                month=r.dateOfExperiment[4:6], 
                day=r.dateOfExperiment[6:8])
        tup['session_suffix'] = os.path.basename(key['session_file']).split('_')[3][8]
        self.insert1(tup)        

In [29]:
Session().populate()

In [34]:
@schema
class PhotoStim(dj.Imported):
    definition = """
    -> Session
    ---
    wavelength : decimal(4,1)
    """
    
    class Location(dj.Part):
        definition = """
        -> PhotoStim
        photostim_loc  : tinyint 
        ---
        area  : varchar(30)
        photostim_x   :  decimal(4,2)    # mm
        photostim_y   :  decimal(4,2)    # mm 
        """
        
    class Method(dj.Part):
        definition = """
        -> PhotoStim
        photostim_method : varchar(255)
        """
        
    def _make_tuples(self, key):
        file = (Session() & key).fetch1['session_file']
        print(file)
            

In [35]:
@schema
class Recording(dj.Computed):
    definition="""
    -> Session()
    ---
    recording_file : varchar(255)
    """
    
    def _make_tuples(self, key):
        a = (Session() & key).fetch1()
        if a['session_suffix']=='.':
            a['session_suffix']=''
        f = os.path.join(
            'data', 'data_structure_ANM{animal:06d}',
            'data_structure_ANM{animal:06d}_{short_date}{session_suffix}.mat').format(
            **a, short_date=''.join(str(a['session_date']).split('-')))
        assert os.path.isfile(f)
        self.insert1(dict(key, recording_file=f))

In [36]:
Recording().populate()

AssertionError: 

In [ ]:
Recording()

# Import units, trials, and spikes

In [ ]:
@schema
class Unit(dj.Imported):
    definition = """
    -> Recording
    unit   :  smallint   # unit number on the array
    ---
    cell_type : varchar(30)
    """
    
    def _make_tuples(self, key):
        f = (Recording() & key).fetch1['recording_file']
        r = scio.loadmat(f, struct_as_record=False, squeeze_me=True)['obj'].eventSeriesHash
        for name, value in zip(r.keyNames, r.value):
            tup = key
            tup['unit'] = int(name[4:])
            tup['cell_type'] = value.cellType if isinstance(value.cellType, str) else ''
            self.insert1(tup)
    
    

In [ ]:
Unit().populate()

In [ ]:
@schema
class Trial(dj.Imported):
    definition = """
    -> Recording
    trial  :  int 
    ---
    start_time : double
    pole_in_time =null:  double
    pole_out_time =null : double
    cue_time =null : double
    good_trial  : tinyint   # change to bool
    photostim_type =null :  tinyint
    """
    
    class Type(dj.Part):
        definition = """
        -> Trial
        trial_type : varchar(12)
        """
        
    def _make_tuples(self, key):
        print(key)
        f = (Recording() & key).fetch1['recording_file']
        obj = scio.loadmat(f, struct_as_record=False, squeeze_me=True)['obj']
        names = ['pole_in_time', 'pole_out_time', 'cue_time', 'good_trial', 'photostim_type'];
        ttype = Trial.Type()
        trial_source = (dict(zip(names, n)) for n in zip(*obj.trialPropertiesHash.value))
        for i, trial in enumerate(trial_source, start=1):
            self.insert1(dict(key, trial=i, start_time=obj.trialStartTimes[i-1], **trial))
            ttype.insert(dict(key, trial=i, trial_type=g) 
                         for g in obj.trialTypeStr[obj.trialTypeMat[:,i-1]>0])

In [ ]:
Trial().populate(reserve_jobs=True, suppress_errors=True)

In [ ]:
@schema
class Spikes(dj.Imported):
    definition = """
    -> Unit
    -> Trial
    ---
    spike_times : longblob    # spikes within trial
    """
    
    @property
    def key_source(self):
        return Recording() & Trial() & Unit()
    
    def _make_tuples(self, key):
        print(key)
        f = (Recording() & key).fetch1['recording_file']
        obj = scio.loadmat(f, struct_as_record=False, squeeze_me=True)['obj']
        for unit_name, value in zip(obj.eventSeriesHash.keyNames, obj.eventSeriesHash.value):
            tup = dict(key, unit=int(unit_name[4:]))
            self.insert(dict(tup, trial=trial, spike_times=value.eventTimes[value.eventTrials==trial]) 
                        for trial in set(value.eventTrials))

In [ ]:
Trial()

In [ ]:
Spikes().populate(suppress_errors=True, reserve_jobs=True)

In [ ]:
dj.ERD(schema).draw()

# Plot results

In [ ]:
# pick a dataset
keys = list((Recording() & Spikes()).fetch.keys())
key = keys[30]

In [ ]:
# plot PSTHs of left (blue) vs right (red) trials
print(key)
good_trials = (Trial() & key & 'good_trial') - (Trial.Type() & 'trial_type in ("LickEarly", "StimTrials")')
left_trials = good_trials & (Trial.Type() & 'trial_type in ("HitL")')
right_trials = good_trials & (Trial.Type() & 'trial_type in ("HitR")')
n_units = len(Unit() & key)
print('Hits: left', len(left_trials), 'right', len(right_trials), 'Units: ', n_units)

ncols = 4
nrows = (n_units + ncols - 1)//ncols
pylab.rcParams['figure.figsize'] = (16, 10)

f, ax = plt.subplots(nrows, ncols)
ax  = ax.flatten()
bins = np.linspace(0,6,6)
x = (bins[:-1]+bins[1:])/2
for i, unit_key in enumerate((Unit() & key).fetch.keys()):
    print(end='.')
    left = (Trial()*Spikes() & unit_key & left_trials).fetch['start_time', 'spike_times']
    if left[0].size:
        left = np.concatenate([spikes-start for start, spikes in zip(*left)])
        ax[i].hist(left, bins, color='blue', alpha=0.5)
    right = (Trial()*Spikes() & unit_key & right_trials).fetch['start_time', 'spike_times']
    if right[0].size:
        right = np.concatenate([spikes-start for start, spikes in zip(*right)])
        ax[i].hist(right, bins, color='red', alpha=0.5)
    ax[i].set_title('Unit {unit}'.format(**unit_key))

# clear unused axes
for i in range(n_units,ncols*nrows):
    ax[i].axis('off')
